In [ ]:
# import
import numpy as np
import pandas as pd
from pathlib import Path
import chardet


In [ ]:
# Create path to CSV
gaming = Path("GamingStudy/Scrub_gaming_data.csv")

In [ ]:
# read in binary mode and detect encoding
# with open("../Scrub gaming data.csv", 'rb') as f:
#     result = chardet.detect(f.read())
    
# encoding = result['encoding']



In [ ]:
# Read data file with Pandas
# gaming_df = pd.read_csv(gaming, encoding = "ISO-8859-1")
gaming_df = pd.read_csv(gaming)
gaming_df

In [ ]:
gaming_df.columns

In [ ]:
clean_game = gaming_df[["S. No.","Game","Platform","Hours","earnings","whyplay","Gender","Age","Work","Degree","Residence","Playstyle","GAD_T","SWL_T","SPIN_T"]]
clean_game

In [ ]:
clean_game.to_csv("clean_game_data.csv")


In [ ]:
# Aliyu's Code Starts Here

In [ ]:
#Aliyu's code stops here

#Amy's code starts here
# Americans vs Europeans: Who plays more (in hours) and how does it compare to their satisfaction with life scores?

In [ ]:
# create a df to see only S. No. (survey number), Residence, and SWL_T (satisfaction with life total score)
residence_df = clean_game[["S. No.","Hours", "Residence","SWL_T"]]

# pull list of countries out of Residence column
residence_df["Residence"].unique()

# residence_df = residence_df.drop(residence_df["Residence"] == 'South Korea', 'Japan', 
#        'Australia', 'Malaysia',
#        'Turkey', 'South Africa', 'India',
#        'Argentina',
#        'Singapore', 'China', 'Unknown', 
#        'Saudi Arabia',
#        'Jordan', 'Brazil', 'Venezuela', 'Tunisia', 'Israel',
#        'Qatar', 'Mexico', 'Philippines',
#        'Egypt', 'New Zealand ', 
#        'Algeria', 'UAE', 'Chile', 'Lebanon',
#        'Thailand', 'Peru', 'Namibia', 'Uruguay', 'Kuwait', 'Bangladesh',
#        'Syria', 'Colombia', 'Dominican Republic', 'Nicaragua',
#        'Hong Kong', 'Vietnam', 'Kazakhstan', 'Guadeloupe',
#        'Grenada', 'Trinidad & Tobago', 'Panama', 'Indonesia',
#        'Puerto Rico', 'Taiwan', 'Costa Rica', 'Belize',
#        'Jamaica', 'Georgia', 'Faroe Islands', 'Guatemala', 'Moldova',
#        'Mongolia', 'Honduras', 'Bahrain', 'Bolivia',
#        'El Salvador', 'Ecuador', 'Pakistan', 'Republic of Kosovo',
#        'St Vincent', 'Brunei', 'Fiji', 'Gibraltar ', 'Palestine')

# drop non-US and non_European countries
# coding help from Xpert Learning Assistant
# European countries reference: https://www.worldometers.info/geography/how-many-countries-in-europe/#google_vignette
patterns_to_drop = ['South Korea', 'Japan', 'Canada', 
       'Australia', 'Malaysia',
       'Turkey', 'South Africa', 'India',
       'Argentina',
       'Singapore', 'China', 'Unknown', 
       'Saudi Arabia',
       'Jordan', 'Brazil', 'Venezuela', 'Tunisia', 'Israel',
       'Qatar', 'Mexico', 'Philippines',
       'Egypt', 'New Zealand ', 
       'Algeria', 'UAE', 'Chile', 'Lebanon',
       'Thailand', 'Peru', 'Namibia', 'Uruguay', 'Kuwait', 'Bangladesh',
       'Syria', 'Colombia', 'Dominican Republic', 'Nicaragua',
       'Hong Kong', 'Vietnam', 'Kazakhstan', 'Guadeloupe',
       'Grenada', 'Trinidad & Tobago', 'Panama', 'Indonesia',
       'Puerto Rico', 'Taiwan', 'Costa Rica', 'Belize',
       'Jamaica', 'Georgia', 'Faroe Islands', 'Guatemala', 'Moldova',
       'Mongolia', 'Honduras', 'Bahrain', 'Bolivia',
       'El Salvador', 'Ecuador', 'Pakistan', 'Republic of Kosovo',
       'St Vincent', 'Brunei', 'Fiji', 'Gibraltar ', 'Palestine']

residence_df = residence_df[~residence_df["Residence"].str.contains('|'.join(patterns_to_drop))]
residence_df                                                            

In [ ]:
# add Region col for US and Eur (only those two)
# assign US or EUR to each country in the Region col
# total up SWL scores for each
# total up hours for each
# pie chart for # players US vs Euro
# bar chart for US/Euro players vs SWL score
# ANOVA analysis?

In [ ]:
#Amy's code stops here
#Andrew's code starts here

In [ ]:
#Anna's code stops here
#Andrew's code starts here